In [7]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import unicodedata
from random import randint
from time import sleep
import re
import math

In [8]:
'''
Função figure_out_target_pages( ) considera o funcionamento da página de resultados do site Reclame Aqui 
e calcula os números corretos de páginas a serem inseridas no URL que contém um 
conjunto de 10 novas reclamações, evitando a coleta repetida.

Recebe como argumento o número de reclamações máximo que se deseja coletar,
a partir do qual calcula as páginas corretas para raspagem. Suporta até o 
número máximo de reclamações registradas no perfil, que devem ser visualizadas
no site.

Parâmetros:
n_of_complaints (int): indica o número de reclamações que se deseja coletar do site

'''

def figure_out_target_pages(n_of_complaints):   
  n_of_pages = math.floor(n_of_complaints / 10)+1
  true_new_complaints_pages = []
  for i in range(0,n_of_pages):
    x = i*10+1
    true_new_complaints_pages.append(x)
  # retorna lista com os números de páginas corretos para raspagem
  return true_new_complaints_pages  

In [9]:

'''
Função scrape_search_results_RA ( ) recebe nome da empresa conforme padrão registrado no URL da sua página de perfil
e número de reclamações a serem coletadas. Chama a função figure_out_target_pages( ) e itera sobre a lista
resultante coletando apenas títulos, datas e links da página de resultados. Retorna um dicionário RA_dict 
com os dados coletados da página de busca.

Parâmetros:
company_name_url (str): string com o nome da empresa tal como escrito na url da página de perfil da empresa no site
n_of_complaints  (int): indica o número de reclamações que se deseja coletar do site

'''

def scrape_search_results_RA (company_name_url, n_of_complaints):
  RA_dict = {
             'Titles' : [],
             'Dates'  : [],
             'Links'  : []
            }

  titles  = []
  dates   = []
  links   = []
  
  target_pages = figure_out_target_pages(n_of_complaints)
  for page in target_pages:
    delay = randint(0,5)
    print(f'Cool down of {delay}.\nScraping page number {page}')

    # Acessa página da empresa e itera sobre os números em target_pages
    # Informa informações de headers para conseguir acesso ao site
    # Gera doc html para raspagem
    url      = f'https://www.reclameaqui.com.br/empresa/{company_name_url}/lista-reclamacoes/?pagina={page}'
    headers  = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0'}
    response = requests.get(url, headers = headers)
    html_doc = BeautifulSoup(response.content, 'html.parser')

    # Testa response e em caso de erro, imprime erro e encerra loop
    if str(response) != '<Response [200]>':
      print(url); print(f'Erro de Request!\n{response}\n Entre em contato se precisa de ajuda.')
      break

    # Em caso de erro inexistente, inicia raspagem
    else:
      cards = html_doc.find_all('div', class_ = 'sc-1pe7b5t-0 iQGzPh')
      for card in cards:
        title = card.find('h4')
        titles.append(title.text)
        date = card.find('span', class_ = 'sc-1pe7b5t-5 bmtSzo')
        dates.append(date.text)
        link = card.find('a')
        links.append(f'https://www.reclameaqui.com.br{link["href"]}')

  # Verifica lengths das listas para evitar erros na criação do data frame
  a = len(titles)
  print(f'Length of titles = {a}')
  list(titles)
  b = len(dates)
  print(f'Length of dates = {b}')
  list(dates)
  c = len(links)
  print(f'Length of links = {b}')
  list(links)
  if a == b and a == c:
    print(f'Listas de dados raspados com o mesmo tamanho. {a} items.')
  else :
    print('Error!\nListas de dados raspados com tamanhos diferentes.\nEntre em contato se precisa de ajuda')

  # Atualiza dicionário com dados raspados e retorna dict
  RA_dict['Titles'] = titles
  RA_dict['Dates']  = dates
  RA_dict['Links']  = links
  return RA_dict


In [11]:
'''
Função scrape_complaints_RA ( ) recebe dicionário gerado por scrape_search_results_RA( ), 
acessa links coletados e raspa texto completo de reclamação do OP e mensagem 
subsequente, podendo esta ser uma réplica, segundo comentário do OP ou nenhuma,
registrando a ausência de mensagem subsequente como 'Não respondida'.

Parâmetros:
RA_dict (dict): recebe a variável que armazena o dicionário produzido pela função scrape_search_results_RA( )

'''

def scrape_complaints_RA(RA_dict):
  links = RA_dict['Links']
  complaints = []
  answers    = []

  for link in links:
    delay = randint(0,5)
    print(f'Cool down of {delay}.\nScraping {link}')
    sleep(delay)

    # Acessa links, informa headers para receber acesso ao site e gera doc html para raspagem
    url      = link
    headers  = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/116.0'}
    response = requests.get(url, headers = headers)
    html_doc = BeautifulSoup(response.content, 'html.parser')

    # Testa response, imprime erro e encerra loop ou procede com a raspagem
    if str(response) != '<Response [200]>':
      print(url); print(f'Erro de Request!\n{response}\n Entre em contato se precisa de ajuda.')
      break
    else:
      complaint_text = html_doc.find('p', class_ = 'lzlu7c-17 cNqaUv').text
      complaints.append(complaint_text)

      # verifica existência de texto de resposta, em caso de inexistência adiciona "Não Respondida" à lista answers
      answer = html_doc.find('p', class_ = 'sc-1o3atjt-4 kBLLZs')
      if answer == None:
        answers.append('Não respondida')
      else:
        answers.append(answer.text)

  # Atualiza dicionário com dados raspados
  RA_dict['Complaints'] = complaints
  RA_dict['Answers']    = answers
  return RA_dict

In [13]:

'''
Função final para o usuário scrape_RA( ), recebe input do usuário, executa as funções
scrape_search_results_RA( ) e scrape_complaints_RA( ), cria df a partir do RA_dict 
gerado por scrape_complaints_RA( ) e salva arquivo no disco ou Google Drive do usuário.

Parâmetros:
company_name_url (str): string com o nome da empresa tal como escrito na url da página de perfil da empresa no site
n_of_complaints  (int): indica o número de reclamações que se deseja coletar do site
file_type        (str): aceita "excel", "csv" ou "csv + excel", indicando formato em que se deseja exportar o df gerado
save_path        (str): indica o diretório em que se deseja salvar o arquivo

'''

def scrape_RA(company_url, n_of_complaints, file_type, save_path):
  company_name_url = company_url.replace('https://www.reclameaqui.com.br/empresa/', '').replace('lista-reclamacoes/', '').replace('/','')
  n_of_complaints  = int(n_of_complaints)
  RA_dict          = scrape_search_results_RA(company_name_url, n_of_complaints)
  RA_dict          = scrape_complaints_RA(RA_dict)
  RA_df            = pd.DataFrame.from_dict(RA_dict)

  if file_type   == 'excel':
    RA_df.to_excel(f'{save_path}\\ReclameAqui_{company_name_url}.xlsx', index=False)
  elif file_type == 'csv':
    RA_df.to_csv(f'{save_path}\\ReclameAqui_{company_name_url}.csv', index=False)
  elif file_type == 'csv + excel':
    RA_df.to_excel(f'{save_path}\\ReclameAqui_{company_name_url}.xlsx', index=False)
    RA_df.to_csv(f'{save_path}\\ReclameAqui_{company_name_url}.csv', index=False)
